In [15]:
# Install libraries if you are running locally or in Colab
# !pip install folium pandas 

import folium
import pandas as pd

# Import folium plugins
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [16]:
import requests
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)
spacex_csv_file = io.BytesIO(response.content)
spacex_df = pd.read_csv(spacex_csv_file)

# Select relevant columns and group by Launch Site
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

In [11]:
# Create the marker cluster
marker_cluster = MarkerCluster()

# Function to assign color based on the value of class column
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

# Add the color column to the dataframe
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Add marker cluster to current site_map
site_map.add_child(marker_cluster)

# For each row in spacex_df, create a marker and add it to the cluster
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=record['Launch Site']
    )
    marker_cluster.add_child(marker)

site_map

In [17]:
# Center the map at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a circle and marker for each launch site
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    
    # Add Circle
    circle = folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(row['Launch Site']))
    
    # Add Marker with label
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

In [18]:
# Create MarkerCluster object
marker_cluster = MarkerCluster()

# Function to assign color based on class (1=green, 0=red)
def assign_marker_color(launch_outcome):
    return 'green' if launch_outcome == 1 else 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
site_map.add_child(marker_cluster)

# Add markers to the cluster
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=record['Launch Site']
    )
    marker_cluster.add_child(marker)

site_map

In [19]:
from math import sin, cos, sqrt, atan2, radians

# Add MousePosition to find coordinates for proximities
site_map.add_child(MousePosition())

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0 # Radius of earth in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon, dlat = lon2 - lon1, lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Example: Distance to Coastline (Update coordinates as needed)
launch_site_lat, launch_site_lon = 28.563197, -80.576820
coastline_lat, coastline_lon = 28.56367, -80.56763
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Draw PolyLine between site and proximity point
lines = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]], weight=1)
site_map.add_child(lines)

site_map